In [1]:
import os

os.environ["SM_FRAMEWORK"] = "tf.keras"  # Set the segmentation_models framework
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Do not use GPU

In [9]:
from sahi.slicing import slice_image
from tqdm import tqdm
from PIL import Image

import segmentation_models as sm
import seaborn as sns
import numpy as np
import glob
import cv2

In [55]:
# Data
DATA_X: str = "x-original"
OUTPUT_FOLDER = f"{DATA_X}-output"

# Training
BATCH_SIZE: int = 8

TRAIN_METRICS: dict[str, str | sm.base.objects.Metric] = {
    "accuracy": "accuracy",
    "dice_loss": sm.losses.dice_loss,
    "precision": sm.metrics.precision,
    "recall": sm.metrics.recall,
    "f1-score": sm.metrics.f1_score,
    "f2-score": sm.metrics.f2_score
}

THRESHOLD_FOR_COUNTING = 0.1

# Parameters for slicing
IMAGE_SIZE = 512
IMAGE_DPI_SCALE = 600/1200    ## TODO this resize needs to be done based on DPI; train data = 600 DPI, test data = 1200 DPI
IMAGE_OVERLAP = 0

MODEL_NAME: str = os.path.join("models", "model.keras")

# To make graphs prettyBATCH_SIZE: int = 8
sns.set_theme()



In [8]:
# Create model
model = sm.Unet("seresnet18", classes=2, encoder_weights=None, input_shape=(512, 512, 3))
model.compile("Adam", sm.losses.DiceLoss(), metrics=list(TRAIN_METRICS.values()))

# Load model if already trained
model.load_weights(MODEL_NAME)

2025-02-19 20:19:46.674488: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-02-19 20:19:46.674512: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: anonymous
2025-02-19 20:19:46.674518: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: anonymous
2025-02-19 20:19:46.674665: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 535.183.1
2025-02-19 20:19:46.674699: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 535.183.1
2025-02-19 20:19:46.674707: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 535.183.1
/home/tadeas/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: Us

In [57]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for image in tqdm(glob.glob(f"{DATA_X}/*.tif")):
    original_image = cv2.resize(cv2.imread(image), None, fx=IMAGE_DPI_SCALE, fy=IMAGE_DPI_SCALE)
    sliced = slice_image(
        image=Image.fromarray(original_image),
        slice_height=IMAGE_SIZE,
        slice_width=IMAGE_SIZE,
        overlap_height_ratio=IMAGE_OVERLAP,
        overlap_width_ratio=IMAGE_OVERLAP
    )

    x = np.array([i["image"] for i in sliced], dtype="float32")
    y = model.predict(x, batch_size=BATCH_SIZE)

    final_image = np.zeros(original_image.shape[:2])

    # Reconstruct final prediction
    # TODO do the overlap, as it may affect the final result
    for n, i in enumerate(sliced):
        px, py = i["starting_pixel"]
        final_image[py:py+IMAGE_SIZE, px:px+IMAGE_SIZE] = y[n, :, :, 1]
        cv2.line(original_image, (px, py), (px + IMAGE_SIZE, py), (0, 255, 0), 1)
        cv2.line(original_image, (px, py), (px, py + IMAGE_SIZE), (0, 255, 0), 1)

    # Count objects in the test and prediction
    contours_pred, _ = cv2.findContours((final_image >= THRESHOLD_FOR_COUNTING).astype("uint8"), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cv2.drawContours(original_image, contours_pred, -1, (0, 0, 255), 2)
    cv2.imwrite(image.replace(DATA_X, OUTPUT_FOLDER) + f"_count={len(contours_pred)}.jpg", original_image)

  0%|          | 0/33 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step


  3%|▎         | 1/33 [00:11<06:02, 11.32s/it]

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step


  6%|▌         | 2/33 [00:22<05:39, 10.94s/it]

...

 94%|█████████▍| 31/33 [05:22<00:20, 10.22s/it]

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step


 97%|█████████▋| 32/33 [05:32<00:10, 10.14s/it]

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step


100%|██████████| 33/33 [05:42<00:00, 10.38s/it]
